In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
!nvidia-smi

Thu Feb  3 19:16:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/HopeCodalab/Hope_ENG_train.csv',names=['text','label'])
test=pd.read_csv('/content/drive/MyDrive/HopeCodalab/Hope_ENG_dev.csv',names=['text','label'])
eval=pd.read_csv('/content/drive/MyDrive/HopeCodalab/Hope_ENG_test.csv',names=['text'])
neweval=pd.read_csv('/content/drive/MyDrive/HopeCodalab/newTest.csv',names=['text'])

In [ ]:
train

,text,label
0,these tiktoks radiate gay chaotic energy and i...,Non_hope_speech
1,@Champions Again He got killed for using false...,Non_hope_speech
2,It's not that all lives don't matter,Non_hope_speech
3,Is it really that difficult to understand? Bla...,Non_hope_speech
4,Whenever we say black isn't that racists? Why...,Non_hope_speech
...,...,...
22735,It's a load of bollocks every life matters sim...,Non_hope_speech
22736,no say it because all lives matter! deku would...,Non_hope_speech
22737,God says her life matters,Non_hope_speech
22738,This video is just shit. A bunch of whiny ass ...,Non_hope_speech


In [ ]:
import re
def simple_text_clean(x):
    # first we lowercase everything
    x = x.lower()
    # remove unicode characters
    x = x.encode('ascii', 'ignore').decode()
    #remove websites
    x = re.sub(r'[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?','',x)
    x = re.sub(r'[^a-zA-z.,!?/;\"\'\s]','',x)#removes special characters except some & remove nums 0-9
    return x

In [ ]:
def changelabel_cleantext(temp):
  temp2=temp.label
  res=pd.DataFrame(temp2)
  for i in res:
      res.loc[res[str(f'{i}')]== 'Non_hope_speech', str(f'{i}')] = 0
      res.loc[res[str(f'{i}')]== 'Hope_speech', str(f'{i}')] = 1
      res[str(f'{i}')]=res[str(f'{i}')].astype(str).astype(int)
  temp['text']=temp['text'].apply(str).apply(lambda x: simple_text_clean(x))
  newdf=pd.concat([temp['text'],res],axis=1)
  return newdf

In [ ]:
train=changelabel_cleantext(train)
test=changelabel_cleantext(test)

In [ ]:
!pip install transformers
from transformers import RobertaModel, RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large', truncation=True, do_lower_case=True)

In [ ]:
# import statistics as st
# max_len = 0
# lens=[]
# sentences=train.text

# # For every sentence...
# for sent in sentences:

#     # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
#     input_ids = tokenizer.encode(sent, add_special_tokens=True)
#     lens.append(len(input_ids))
#     # Update the maximum sentence length.
#     max_len = max(max_len, len(input_ids))

# print('Max sentence length: ', max_len)
# print('Max sentence length: ', st.mean(lens))

In [ ]:
class RobertaData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = self.data.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
training_set = RobertaData(train, tokenizer, 22)
testing_set = RobertaData(test, tokenizer, 22)

In [ ]:
train_params = {'batch_size': 16,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 16,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
robertamodel = RobertaModel.from_pretrained('roberta-large',output_hidden_states=True)
# for p in robertamodel.parameters():
#     p.requires_grad = True

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = robertamodel
        self.pre_classifier = torch.nn.Linear(1024, 1024)
        self.bn=torch.nn.LayerNorm(1024)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(1024, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        bnorm = self.bn(pooler)
        # pooler = torch.nn.ReLU()(pooler)
        tanlayer =  torch.nn.Tanh()(bnorm)
        tanlayer = self.dropout(tanlayer)
        output = self.classifier(tanlayer)
        return output

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
model = RobertaClass()
model.to(device)
model.cuda()

RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNo

In [ ]:
# # Creating the loss function and optimizer
# criterion = torch.nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-5)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
import tqdm as tqdm

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

In [ ]:
from tqdm import tqdm, trange

In [ ]:
!pip install madgrad
from madgrad import MADGRAD

In [ ]:
def evaluate(model,a, tokenizer, criterion, dataloader, tres = 0.5): 
    
    # Eval!
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    proba = None
    out_label_ids = None
    for batch in a(dataloader):
        model.eval()
        with torch.no_grad():
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
            targets = batch['targets']
            targets=targets.view(targets.size(0),-1)
            labels=targets.to(device, dtype = torch.float32)
            outputs = model(ids, mask, token_type_ids)
            logits = outputs  # model outputs are always tuple in transformers (see doc)
            tmp_eval_loss = criterion(logits, labels) 
            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = torch.sigmoid(logits).detach().cpu().numpy() > tres
            proba = torch.sigmoid(logits).detach().cpu().numpy()
            out_label_ids = labels.detach().cpu().numpy()
        else:            
            preds = np.append(preds, torch.sigmoid(logits).detach().cpu().numpy() > tres, axis=0)
            proba = np.append(proba, torch.sigmoid(logits).detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, labels.detach().cpu().numpy(), axis=0)
    
    eval_loss = eval_loss / nb_eval_steps

    result = {
        "loss": eval_loss,
        "accuracy": accuracy_score(out_label_ids, preds),
        "AUC": roc_auc_score(out_label_ids, proba),
        "micro_f1": f1_score(out_label_ids, preds, average="micro"),
        "prediction": preds,
        "labels": out_label_ids,
        "proba": proba
    }
    
    return result

In [ ]:
def save_checkpoint(save_path, model, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')
    
def load_checkpoint(load_path, model):
    
    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    return model

In [ ]:
num_labels = 1
gradient_accumulation_steps = 20
max_seq_length = 22
max_grad_norm = 0.5
train_batch_size = 16
eval_batch_size = 16
num_train_epochs = 5

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch import nn
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ["bias", 
            "LayerNorm.weight"
           ]
weight_decay = 0.0005

optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

t_total = (len(training_loader) // gradient_accumulation_steps) * num_train_epochs
warmup_steps = t_total // 10

optimizer = MADGRAD(optimizer_grouped_parameters, lr=2e-4)

scheduler = get_linear_schedule_with_warmup(
        optimizer, warmup_steps, t_total
    )

criterion = nn.BCEWithLogitsLoss()

In [ ]:
optimizer_step = 0
gradient_accumulation_steps=20
max_grad_norm=1
global_step = 0
train_step = 0
tr_loss, logging_loss = 0.0, 0.0
best_valid_f1 = 0.73
global_steps_list = []
train_loss_list = []
val_loss_list = []
val_acc_list = []
val_auc_list = []
eval_every = len(training_loader) // 7
running_loss = 0
file_path='/content/drive/MyDrive/RajWorkspace/mytestresults'

model.zero_grad()

for i in range(num_train_epochs):
    print("Epoch", i+1, f"from {num_train_epochs}")
    whole_y_pred=np.array([])
    whole_y_t=np.array([])
    for step, batch in enumerate(tqdm(training_loader)):
        model.train()
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
        targets = batch['targets']
        targets=targets.view(targets.size(0),-1)
        labels=targets.to(device, dtype = torch.float32)
        outputs = model(ids, mask, token_type_ids)
        logits = outputs   # model outputs are always tuple in transformers (see doc)
        loss = criterion(logits, labels) 
        
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
            
        loss.backward()
        
        tr_loss += loss.item()
        running_loss += loss.item()
        global_step += 1
        
        if (step + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()  # Update learning rate schedule         
            
            optimizer_step += 1
            optimizer.zero_grad()   
                        
        if (step + 1) % eval_every == 0:
            
            average_train_loss = running_loss / eval_every
            train_loss_list.append(average_train_loss)
            global_steps_list.append(global_step)
            running_loss = 0.0  
            
            val_result = evaluate(model,tqdm, tokenizer, criterion, testing_loader)
            
            # checkpoint
            if val_result['micro_f1'] > best_valid_f1:
                best_valid_auc = val_result['AUC']
                val_loss = val_result['loss']
                val_acc = val_result['accuracy']
                best_valid_f1=val_result['micro_f1']
                model_path = f'{file_path}/model-auc{best_valid_auc:.3f}-loss{val_loss:.3f}-acc{val_acc:.3f}.pt'
                print(f"AUC improved, so saving this model")  
                save_checkpoint(model_path, model, val_result['loss'])              
            
            print(  "Val loss:", f"{val_result['loss']:.4f}",
                    "Val acc:", f"{val_result['accuracy']:.4f}",
                    "Val f1:", f"{val_result['micro_f1']:.4f}",
                    "AUC:", f"{val_result['AUC']:.4f}")   
    print('\n')

Epoch 1 from 5


  0%|          | 0/1422 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 178/178 [00:09<00:00, 18.24it/s]


AUC improved, so saving this model


 14%|█▍        | 203/1422 [00:46<1:32:11,  4.54s/it]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
 14%|█▍        | 204/1422 [00:46<1:05:28,  3.23s/it]

Model saved to ==> /content/drive/MyDrive/RajWorkspace/mytestresults/model-auc0.402-loss0.354-acc0.904.pt
Val loss: 0.3543 Val acc: 0.9043 Val f1: 0.9043 AUC: 0.4015


...

Val loss: 0.3177 Val acc: 0.9043 Val f1: 0.9043 AUC: 0.4914


 86%|████████▌ | 1218/1422 [04:03<10:30,  3.09s/it]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
 86%|████████▌ | 1219/1422 [04:03<07:28,  2.21s/it]

Val loss: 0.3179 Val acc: 0.9043 Val f1: 0.9043 AUC: 0.5284


100%|█████████▉| 1421/1422 [04:43<00:03,  3.11s/it]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1422/1422 [04:43<00:00,  5.01it/s]

Val loss: 0.3167 Val acc: 0.9043 Val f1: 0.9043 AUC: 0.5265




In [ ]:
def final_prediction(model, dataloader): 
        preds = None
        proba = None
        all_ids = None
        for batch in tqdm(dataloader):
            model.eval()
            with torch.no_grad():
                ids = batch['ids'].to(device, dtype = torch.long)
                mask = batch['mask'].to(device, dtype = torch.long)
                token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
                targets = batch['targets']
                targets=targets.view(targets.size(0),-1)
                labels=targets.to(device, dtype = torch.float32)
                outputs = model(ids, mask, token_type_ids)
                logits = outputs
            if preds is None:
                # all_ids = ids.detach().cpu().numpy()
                preds = torch.sigmoid(logits).detach().cpu().numpy() > 0.5
                proba = torch.sigmoid(logits).detach().cpu().numpy()            
            else:  
                # all_ids = np.append(all_ids, ids.detach().cpu().numpy(), axis=0)
                preds = np.append(preds, torch.sigmoid(logits).detach().cpu().numpy() > 0.5, axis=0)
                proba = np.append(proba, torch.sigmoid(logits).detach().cpu().numpy(), axis=0)

        result = {
            # "ids": all_ids,
            "preds": preds,
            "probs": proba,
        }

        return result

In [ ]:
path='/content/drive/MyDrive/RajWorkspace/mytestresults/model-auc0.913-loss0.186-acc0.935.pt'
bestmodel_final=load_checkpoint(path,model)

Model loaded from <== /content/drive/MyDrive/RajWorkspace/mytestresults/model-auc0.913-loss0.186-acc0.935.pt


In [ ]:
eval_params = {'batch_size': 1,
                'shuffle': False,
                'num_workers': 0
                }

eval_loader = DataLoader(testing_set, **eval_params)

In [ ]:
results_final=final_prediction(model, eval_loader)

  0%|          | 0/2841 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 2841/2841 [00:49<00:00, 57.96it/s]


In [ ]:
resf=pd.DataFrame(results_final['preds'])
resf.columns=['label']
resf.loc[resf['label']== True, "label"] = 1
resf.loc[resf['label']== False, "label"] = 0
resf['label']=resf['label'].astype(str).astype(int)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test.label, resf.label))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      2569
           1       0.00      0.00      0.00       272

    accuracy                           0.90      2841
   macro avg       0.45      0.50      0.47      2841
weighted avg       0.82      0.90      0.86      2841



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
resf.value_counts()

label
0        2619
1         222
dtype: int64